# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.122124e+02     1.413665e+00
 * time: 0.7630660533905029
     1     1.038343e+01     8.676847e-01
 * time: 2.4034430980682373
     2    -1.178435e+01     1.039838e+00
 * time: 3.0885281562805176
     3    -3.434635e+01     7.476202e-01
 * time: 4.042011976242065
     4    -4.780346e+01     6.008637e-01
 * time: 5.017481088638306
     5    -5.690138e+01     2.227735e-01
 * time: 5.99259614944458
     6    -5.961475e+01     1.958275e-01
 * time: 6.629975080490112
     7    -6.073288e+01     9.667213e-02
 * time: 7.270071029663086
     8    -6.122882e+01     3.878293e-02
 * time: 7.909970045089722
     9    -6.152225e+01     6.665616e-02
 * time: 8.671207189559937
    10    -6.170901e+01     2.495934e-02
 * time: 9.311475038528442
    11    -6.186819e+01     2.433724e-02
 * time: 9.952018022537231
    12    -6.198292e+01     2.168103e-02
 * time: 10.598251104354858
    13    -6.204279e+01     2.035068e-02
 * time: 11.630211114883423
   

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671062
    AtomicLocal         -18.8557717
    AtomicNonlocal      14.8522691
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485369 
    Xc                  -19.3336818

    total               -62.261666452064
